# **Analiza Parkrun Polska - pobranie danych**

In [68]:
from urllib.request import Request, urlopen, urlretrieve
import json
import pandas as pd
from bs4 import BeautifulSoup
import time
from utils import db, osm, time_format
from tqdm import tqdm

## Lokalizacje

In [52]:
url = "https://images.parkrun.com/events.json"
filepath = "data/events.json"

urlretrieve(url, "data/events.json")

with open(filepath) as file:
    events_json = json.load(file)
    

### Czyszczenie danych

In [53]:
events = pd.json_normalize(events_json["events"]["features"])
events.head()

,id,type,geometry.type,geometry.coordinates,properties.eventname,properties.EventLongName,properties.EventShortName,properties.LocalisedEventLongName,properties.countrycode,properties.seriesid,properties.EventLocation
0,193,Feature,Point,"[153.051636, -27.471134]",newfarm,New Farm parkrun,New Farm,None,3,1,"New Farm, Brisbane"
1,205,Feature,Point,"[153.428879, -27.972706]",mainbeach,Main Beach parkrun,Main Beach,None,3,1,"Main Beach, Goldcoast"
2,349,Feature,Point,"[153.522781, -28.166758]",kirra,Kirra parkrun,Kirra,None,3,1,"Kirra Beach, Gold Coast QLD, Australia"
3,365,Feature,Point,"[153.281709, -27.885247]",coomera,Coomera parkrun,Coomera,None,3,1,Tallowwood Park
4,417,Feature,Point,"[153.177898, -27.442933]",wynnum,Wynnum parkrun,Wynnum,None,3,1,"Wynnum Manly Foreshore, QLD"


In [54]:
def copy_df(df: pd.DataFrame) -> pd.DataFrame:
    return df.copy()


def filter_countrycode(df: pd.DataFrame, country_code: int) -> pd.DataFrame:
    df = df[df["properties.countrycode"] == country_code]
    return df


def select_columns(df: pd.DataFrame) -> pd.DataFrame:
    columns = [
        "properties.eventname",
        "properties.EventShortName",
        "geometry.coordinates",
    ]

    df = df[columns]
    return df


def rename_columns(df: pd.DataFrame) -> pd.DataFrame:

    df = df.rename(
        columns={
            "properties.eventname": "name",
            "properties.EventShortName": "short_name",
            "geometry.coordinates": "coordinates",
        }
    )

    return df


def extract_coordinates(df: pd.DataFrame) -> pd.DataFrame:
    df = df.apply(osm.split_coordinates, axis=1)
    df = df.drop("coordinates", axis=1)
    return df


def fetch_location_info(df: pd.DataFrame) -> pd.DataFrame:
    df = df.apply(osm.assign_state_city, axis=1)
    return df


In [55]:
events_pl = (
    events
    .pipe(filter_countrycode, 74)
    .pipe(select_columns)
    .pipe(rename_columns)
    .pipe(extract_coordinates)
    .pipe(fetch_location_info)
    .reset_index(drop=True)
)

events_pl.head()

,name,short_name,latitude,longitude,state,location
0,gdynia,Gdynia,54.502632,18.558699,pomorskie,Gdynia
1,gdansk,Gdańsk,54.414280,18.601700,pomorskie,Gdańsk
2,lodz,Łódź,51.752244,19.441498,łódzkie,Łódź
3,poznan,Poznań,52.418883,16.933289,wielkopolskie,Poznań
4,warszawapraga,Warszawa-Praga,52.241493,21.050963,mazowieckie,Warszawa


In [56]:
events_pl.to_csv("data/locations.csv", index=False)

In [57]:
credentials = db.config()
conn = db.create_connection(credentials=credentials, dbname='parkrun')

### Zapisanie wyników do lokalnej bazy danych PostgreSQL

In [59]:
location_table_sql = """
CREATE TABLE locations (
    id serial PRIMARY KEY,
    name VARCHAR(100),
    short_name VARCHAR(100),
    latitude float,
    longitude float, 
    state varchar(100), 
    location VARCHAR(100)
)"""

db.create_table(conn, location_table_sql)

In [60]:
insert_locations_sql = """
INSERT INTO 
    locations (name, short_name, latitude, longitude, state, location) 
VALUES (%s, %s, %s, %s, %s, %s)
"""

iterable = events_pl.itertuples(index=False, name=None)

db.batch_insert_values(conn, insert_locations_sql, iterable)

## Wyniki

In [61]:
create_events_sql =  """
    CREATE TABLE events (
    id serial PRIMARY KEY,
    event_number int,
    date date,
    participants int,
    volunteers int,
    best_male_time int,
    best_male_time_str VARCHAR(10),
    best_female_time int,
    best_female_time_str VARCHAR(10),
    location_id int REFERENCES locations (id)
    )"""

db.create_table(conn, create_events_sql)

In [66]:
with conn.cursor() as cur:
    cur.execute("SELECT id, name FROM locations")
    locations = cur.fetchall()
    cur.close()

In [65]:
for location_id, location in locations:
    req = Request(f"https://www.parkrun.pl/{location}/results/eventhistory/")
    req.add_header('User-Agent', "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36")
    content = urlopen(req).read()
    
    soup = BeautifulSoup(content, "html.parser")
    tbody = soup.find("tbody")

    events = []

    for row in tbody.find_all("tr"):
        event = row.attrs
        events.append(
            (
                location_id,
                int(event["data-parkrun"]),
                event["data-date"],
                int(event["data-finishers"]),
                int(event["data-volunteers"]),
                time_format.string_to_seconds(event["data-maletime"]),
                time_format.format_result_mm_ss(event["data-maletime"]),
                time_format.string_to_seconds(event["data-femaletime"]),
                time_format.format_result_mm_ss(event["data-femaletime"])
            )
        )

    events_sql = """
    INSERT INTO 
        events 
        (location_id, event_number, date, participants, volunteers, best_male_time, best_male_time_str, best_female_time, best_female_time_str) 
        VALUES 
        (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    db.batch_insert_values(conn, sql=events_sql, iterable=events)

    time.sleep(2)
    

In [72]:
with conn.cursor() as cur:
    cur.execute("""
SELECT e.id, l.name, event_number
FROM events e
LEFT JOIN locations l
ON e.location_id = l.id
WHERE l.name in (
SELECT name
FROM (
	SELECT l.name, count(*)
	FROM events e
	LEFT JOIN locations l
	ON e.location_id = l.id
	GROUP BY l.name
	ORDER BY COUNT DESC
	LIMIT 5
) a);

                """)
    events_history = cur.fetchall()
    cur.close()

In [73]:
create_events_sql =  """
    DROP TABLE IF EXISTS results;
    CREATE TABLE results (
    id serial PRIMARY KEY,
    position int,
    gender varchar(10),
    agegroup varchar(20),
    time int,
    time_str VARCHAR(10),
    achievement varchar(100),
    event_id int REFERENCES events (id)
    )"""

db.create_table(conn, create_events_sql)

In [74]:
for event_id, location, event_number in events_history:
    req = Request(f"https://www.parkrun.pl/{location}/results/{event_number}/")
    req.add_header('User-Agent', "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36")
    content = urlopen(req).read()
    
    soup = BeautifulSoup(content, "html.parser")
    tbody = soup.find("tbody")

    events = []

    for row in tbody.find_all("tr"):
        event = row.attrs
        
        if event["data-name"] != "Nieznan(a)y":
        
            time_v = None
            time_col = row.find_all("td")[-1]
            
            if time_col:
                time_v = time_col.find("div", class_='compact').get_text()
            
            events.append(
                (   
                    event_id,
                    int(event["data-position"]),
                    event["data-gender"],
                    event["data-agegroup"],
                    time_format.string_to_seconds_results(time_v),
                    time_v,
                    event["data-achievement"]
                )
            )


    events_sql = """
    INSERT INTO 
        results 
        (event_id, position, gender, agegroup, time, time_str, achievement) 
        VALUES 
        (%s, %s, %s, %s, %s, %s, %s)
    """
    
    db.batch_insert_values(conn, sql=events_sql, iterable=events)

    time.sleep(2)